In [1]:
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain_community.document_loaders import PyPDFLoader
import subprocess
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import subprocess
import shlex

@tool
def send_email(destination_email, message_content, subject_content):
    "This function takes in destination email and the contents and sends out an email."
    # Sender email address
    sender_email = "lucasenkrateia@gmail.com"

    # Email Subject
    subject = subject_content

    # Email body
    body = message_content

    # SMTP server credentials
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # For starttls
    smtp_user = sender_email
    smtp_password = os.environ["GMAIL_PWD"]

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = destination_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Convert message to string for sending
    text = message.as_string()

    # Send email
    try:
        # Connect to the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(smtp_user, smtp_password)
        
        # Send the email
        server.sendmail(sender_email, destination_email, text)
        print("Email sent successfully")
    except Exception as e:
        # Print any error messages to stdout
        print(f"Error: {e}")
    finally:
        server.quit()


@tool
def execute_bash_command(command):
    """Executes a bash command."""
    result = subprocess.run(command, shell=True)
    output = result.stdout
    return output

@tool
def read_txt_tool(file_path):
    """Reads contents of a .txt file."""
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

@tool
def read_pdf_and_save_to_txt(pdfpath, destinationpath):
    """Reads a PDF file and returns the text content."""

    escaped_filepath = shlex.quote(pdfpath)  # Escape the filepath to use in the shell command
    escaped_txtpath = shlex.quote(destinationpath)

    # Use the shell parameter to interpret the command correctly
    subprocess.run(f"pdftotext {escaped_filepath} && cat $(echo {escaped_filepath} | sed 's/\\.pdf$/.txt/') ", shell=True)

@tool
def summarize_pdf(pdfpath):
    """Summarizes the contents of a pdf"""
    loader = PyPDFLoader(pdfpath)
    documents = loader.load()
    contents = ""
    for doc in documents:
        contents += doc.page_content + "\n"
    
    summary = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125").invoke(f"Summarize this content into simplified bullet points: {contents}")
    
    return summary.content
    
tools = [execute_bash_command, send_email, read_txt_tool, summarize_pdf]

In [2]:
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4-0125-preview")

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [3]:
agent_executor.invoke({"input": "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `execute_bash_command` with `{'command': 'echo "1. Automate daily report generation for business analytics.\n2. Develop a script to auto-respond to customer service emails based on keywords.\n3. Create a system to monitor and alert for server downtime.\n4. Build a tool to automate social media posts across various platforms.\n5. Design a program to automatically sort and organize digital files.\n6. Implement a workflow to automate data backup and recovery processes.\n7. Develop a bot to automate routine checks on website performance.\n8. Create a script to automate the installation and setup of development environments.\n9. Build an application to automate expense tracking and budgeting.\n10. Design a system to automatically update software and patch security vulnerabilities." > tasks-for-lucas.txt'}`


None
Invoking: `read_txt_tool` with `{'file_path': 'tasks-for-lucas.txt'}`


1. Automate daily report generation for business analytic

{'input': "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
 'agent_scratchpad': [],
 'output': "I've successfully created a file named `tasks-for-lucas.txt` containing 10 interesting automation tasks and sent an email to `lucasbnsoares@hotmail.com` with the top 3 tasks from that file."}

In [ ]:
agent_executor.invoke({"input": "Create a folder named `lucas-test`. Inside that folder create a file named lucas-test.txt containing a random quote from a famous philosopher. Send an email to '''lucasbnsoares@hotmail.com''' with that quote.",
                             "agent_scratchpad": []})

agent_executor.invoke({"input": """1. Read the contents of this pdf: research report located in /Users/greatmaster/Desktop/projects/research_reports/prompt_engineering_techniques.pdf and save it to ./paper-output.txt. 2. Read the contents of the .txt file. 3. Send the output '''lucasbnsoares@hotmail.com'''""",
                                "agent_scratchpad": []})

In [ ]:
# Let's add a summary to this pipeline.

# Starts making mistakes when I add just summarization to the prompt, and the error is not even at that level.
# So just saying summarize makes the model produce errors in other parts of the pipeline that were working fine previously.
agent_executor.invoke({"input": """1. Summarize this pdf: /Users/greatmaster/Desktop/projects/research_reports/prompt_engineering_techniques.pdf contents to a concise instructive paragraph. 2. Send the output to '''lucasbnsoares@hotmail.com'''""",
                                "agent_scratchpad": []})